In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

from glob import glob
import seaborn as sns

import numpy as np
import pandas as pd

import warnings

from xskillscore import crps_gaussian, rmse

In [2]:
SECONDS_IN_YEAR = 60*60*24*365 #s
convert = lambda x: x * SECONDS_IN_YEAR * 1e-12 # kg -> Gt

# data_path = "F:\\Local Data\\ClimateBench\\"
data_path = '/Users/watson-parris/Local Data/ClimateBench/train_val/'


# Only future scenarios for now
inputs = glob(data_path + "inputs_s*.nc")

def get_rmse(truth, pred):
    return np.sqrt(((truth-pred)**2))

def global_mean(ds):
    weights = np.cos(np.deg2rad(ds.lat))
    return ds.weighted(weights).mean(['lat', 'lon'])


In [4]:
variables = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']

Y = xr.open_dataset(data_path + 'outputs_ssp245.nc').sel(time=slice(2080, 2100))

# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

gp_predictions = global_mean(xr.merge([{v: xr.open_dataarray(data_path +'/revisions_1/'+ "climatebench-gp-posterior-mean-{}-test-2019-2100.nc".format(v))} for v in variables]).sel(time=slice(2080, 2100)))
gp_predictions_std = global_mean(xr.merge([{v: xr.open_dataarray(data_path +'/revisions_1/'+ "climatebench-gp-posterior-std-{}-test-2019-2100.nc".format(v))} for v in variables]).sel(time=slice(2080, 2100)))

In [9]:
CRPS = pd.DataFrame({
    "Gaussian Process": {v: crps_gaussian(Y[v], gp_predictions[v], gp_predictions_std[v]).data for v in variables} 
})
CRPS.T.style.format("{:.4f}")

,diurnal_temperature_range,pr,pr90,tas
Gaussian Process,0.4765,0.3601,1.0753,1.0029
